In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

In [9]:
df = pd.read_csv("Downloads/Vehicle Risk Prediction Dataset.csv")



# Encode categorical features using LabelEncoder
le_visibility = LabelEncoder()
le_road_surface_conditions = LabelEncoder()
le_weather = LabelEncoder()
le_traffic_density = LabelEncoder()
le_road_hazards = LabelEncoder()
#le_time_of_day = LabelEncoder()
le_fatigue_level = LabelEncoder()
le_medical_condition = LabelEncoder()
le_speeding = LabelEncoder()
le_light= LabelEncoder()
#le_road_type=LabelEncoder()
#le_landscape=LabelEncoder()

df['visibility_n'] = le_visibility.fit_transform(df['Visibility'])
df['road_surface_conditions_n'] = le_road_surface_conditions.fit_transform(df['Road_Surface_Conditions'])
df['weather_n'] = le_weather.fit_transform(df['Weather'])
df['traffic_density_n'] = le_traffic_density.fit_transform(df['Traffic_Density'])
df['road_hazards_n'] = le_road_hazards.fit_transform(df['Road_Hazards'])
#df['time_of_day_n'] = le_time_of_day.fit_transform(df['Time_of_Day'])
df['fatigue_level_n'] = le_fatigue_level.fit_transform(df['Fatigue_Level'])
df['medical_condition_n'] = le_medical_condition.fit_transform(df['Medical_Condition'])
df['speeding_n'] = le_speeding.fit_transform(df['Speeding'])
df['light_condition']=le_light.fit_transform(df['Light_Conditions'])
#df['roadtype'] = le_road_type.fit_transform(df['Road_Type'])
#df['landscape_n']=le_landscape.fit_transform(df['Landscape'])

df = df.drop(['Light_Conditions', 'Road_Type', 'Landscape', 'Visibility', 'Road_Surface_Conditions', 'Weather', 'Traffic_Density', 'Road_Hazards', 'Time_of_Day', 'Fatigue_Level', 'Medical_Condition', 'Speeding','Driver_Age','Last_Service_Months_Ago'], axis='columns')



In [10]:
X = df.drop('Risk_Score', axis=1)
y = df['Risk_Score'].apply(lambda x: 1 if x > 50 else 0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
df.head()


,Driver_Experience,Driver_Awake_Time,Vehicle_Speed_Ratio,Cargo_Load,Temperature,Risk_Score,visibility_n,road_surface_conditions_n,weather_n,traffic_density_n,road_hazards_n,fatigue_level_n,medical_condition_n,speeding_n,light_condition
0,1,5.5,1.02,88,22,10.4,0,0,1,1,3,1,5,0,2
1,1,5.5,1.02,88,22,10.4,0,0,1,1,3,1,5,0,2
2,1,5.5,1.02,88,22,10.4,0,0,1,1,3,1,5,0,2
3,3,5.9,1.09,167,25,14.7,0,0,1,1,3,1,5,0,2
4,3,5.9,1.09,167,25,14.7,0,0,1,1,3,1,5,0,2


In [11]:

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create Logistic Regression model with increased max_iter
logreg = LogisticRegression(max_iter=1000, random_state=42)

# Define hyperparameters for tuning
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}

# Use GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(logreg, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_scaled, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Train the model with the best hyperparameters
best_logreg = LogisticRegression(C=best_params['C'], max_iter=1000, random_state=42)
best_logreg.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = best_logreg.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Best Hyperparameters:", best_params)
print("\nAccuracy:", accuracy)
print("\nConfusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)

Best Hyperparameters: {'C': 1}

Accuracy: 1.0

Confusion Matrix:
 [[1168    0]
 [   0 1832]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1168
           1       1.00      1.00      1.00      1832

    accuracy                           1.00      3000
   macro avg       1.00      1.00      1.00      3000
weighted avg       1.00      1.00      1.00      3000

